In [47]:
def simulate_clock(cycles, period=1, clock = 0, prev_clock = 1):
        if clock == 1 and prev_clock == 0:
            posedgeClk = True
        elif clock == 0 and prev_clock == 1:
            posedgeClk = False
        
        return posedgeClk

In [ ]:
for cycle in range(cycles):
        prev_clock = clock
        clock = 1 - clock 

In [64]:
class ProgramCounter:
    def __init__(self):
        self.pc = 0

    def updatePc(self, new_pc, rst, hold, posedgeClk):
        if rst and posedgeClk:
            self.pc = 0
        elif not hold and posedgeClk:
            self.pc = new_pc

    def __repr__(self):
        return f"PC: {self.pc}"


class InstructionMemory:
    def __init__(self, instructions, cycle):
        self.instructions = instructions
        self.cycle = cycle

    def readInstruction(self, address, rst, posedgeClk):
        if address < len(self.instructions) and not rst and posedgeClk:
            return self.instructions[address]
        else:
            return 0


class Fetch:
    def __init__(self, PCsrc, jr, jump, hold,
                 reg1Addr, jaddress, adderResult, cycle, rst, PCPlus1, posedgeClk):
        self.rst = rst
        self.cycle = cycle
        self.PCsrc = PCsrc
        self.jr = jr
        self.jump = jump
        self.hold = hold
        self.reg1Addr = reg1Addr
        self.jaddress = jaddress
        self.adderResult = adderResult
        self.posedgeClk = posedgeClk

        # Compute addresses
        self.branchAddress = adderResult if PCsrc else PCPlus1
        self.jrAddress = reg1Addr if jr else jaddress
        self.nextPC = self.jrAddress if jump else self.branchAddress

        # Initialize Program Counter
        self.pCounter = ProgramCounter()
        self.pCounter.updatePc(self.nextPC, rst, False, self.posedgeClk)
        self.PC = self.pCounter.pc
        self.PCPlus1 = self.PC + 1 if not hold else self.pCounter.pc

        # Load instructions from memory
        instructions = self.extractInstructions("imem.txt")
        self.imemory = InstructionMemory(instructions, cycle)
        self.currentInstruction = self.imemory.readInstruction(self.pCounter.pc, rst, posedgeClk = self.posedgeClk)
    
    @staticmethod
    def extractInstructions(file_path):
        binary_instructions = []
        with open(file_path, "r") as file:
            for line in file:
                parts = line.split(":")
                if len(parts) > 1:
                    binary_value = parts[1].strip().rstrip(";")
                    binary_instructions.append(int(binary_value,2))
        return binary_instructions

    def getInstruction(self):
        return bin(self.currentInstruction)

    def getPCPlus1(self):
        return self.PCPlus1


In [67]:
# Example Usage
clk = 5
fetch_unit = Fetch(
    PCsrc=True, jr=False, jump=False, hold=False,
    reg1Addr=5, jaddress=10, adderResult=15,
    cycle = clk, rst=False, PCPlus1 = 7, posedgeClk=1
)

print("Next Instruction:", fetch_unit.getInstruction())
print("PC+1:", fetch_unit.getPCPlus1())
print("PC:", fetch_unit.PC)
print("cycle:", fetch_unit.cycle)


Next Instruction: 0b100000000000000000000000000000
PC+1: 16
PC: 15
cycle: 5


In [139]:
class ControlUnit:
    _RType = 0x0 
    _addi = 0x8 
    _ori_ = 0xd
    _xori_ = 0xe
    _andi_ = 0xc
    _slti_ = 0xa
    _lw = 0x23 
    _sw = 0x2b 
    _beq = 0x4
    _j_ = 0x2
    _jal_ = 0x3
    _bne_ = 0x5
    _add_ = 0x20
    _sub_ = 0x22
    _and_ = 0x24
    _or_ = 0x25
    _slt_ = 0x2a
    _xor_ = 0x26
    _nor_ = 0x27
    _sll_ = 0x0
    _srl_ = 0x2 
    _jr_ = 0x8    

    def __init__(self, opCode, funct):
        self.opCode = opCode
        self.funct = funct
        self.RegDst = 0b0
        self.Branch = 0b0
        self.MemReadEn = 0b0
        self.MemtoReg = 0b0
        self.MemWriteEn = 0b0
        self.RegWriteEn = 0b0
        self.ALUSrc = 0b0
        self.ALUOp = 0b0
        self.bne = 0b0
        self.jump = 0b0
        self.jal = 0b0
        self.jr = 0b0

    def execute(self):
        if self.opCode == ControlUnit._RType:
            if self.funct == ControlUnit._add_:
                self.ALUOp = 0b0000
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._sub_:
                self.ALUOp = 0b0001
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._and_:
                self.ALUOp = 0b0010
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._or_:
                self.ALUOp = 0b0011
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._slt_:
                self.ALUOp = 0b0100
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._xor_:
                self.ALUOp = 0b0101
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._nor_:
                self.ALUOp = 0b0110
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._sll_:
                self.ALUOp = 0b0111
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._srl_:
                self.ALUOp = 0b1000
                self.RegDst = 0b1
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b1
                self.ALUSrc = 0b0
            elif self.funct == ControlUnit._jr_:
                self.RegDst = 0b0
                self.Branch = 0b0
                self.MemReadEn = 0b0
                self.MemtoReg = 0b0
                self.ALUOp = 0b0000
                self.MemWriteEn = 0b0
                self.RegWriteEn = 0b0
                self.ALUSrc = 0b0
                self.jump = 0b1
                self.bne = 0b0
                self.jal = 0b0
                self.jr = 0b1
        elif self.opCode == ControlUnit._addi:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0000
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b1
            self.ALUSrc = 0b1
        elif self.opCode == ControlUnit._lw:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b1
            self.MemtoReg = 0b1
            self.ALUOp = 0b0000
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b1
            self.ALUSrc = 0b1
        elif self.opCode == ControlUnit._sw:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0000
            self.MemWriteEn = 0b1
            self.RegWriteEn = 0b0
            self.ALUSrc = 0b1
        elif self.opCode == ControlUnit._beq:
            self.RegDst = 0b0
            self.Branch = 0b1
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0001
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b0
            self.ALUSrc = 0b0
            self.jump = 0b0
            self.bne = 0b0
            self.jal = 0b0
            self.jr = 0b0
        elif self.opCode == ControlUnit._ori_:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0011
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b1
            self.ALUSrc = 0b1
        elif self.opCode == ControlUnit._xori_:
            self.RegDst = 0b0
            self.Branch = 0b0
            self.MemReadEn = 0b0
            self.MemtoReg = 0b0
            self.ALUOp = 0b0101
            self.MemWriteEn = 0b0
            self.RegWriteEn = 0b1
            self.ALUSrc = 0b1


In [142]:
cu = ControlUnit(opCode=0x0, funct=0x20)
cu.execute()
print(cu.RegWriteEn)

1


In [144]:
class RF:
    def __init__(self):
        self.registers = [0]*32
        self.readData1 = 0
        self.readData2 = 0
    def writeRegister(self, rst, writeRegister, writeData, writeEnable):
        if rst:
            self.registers = [0]*32
        elif writeEnable:
            self.registers[writeRegister] = writeData
    def readRegisters(self, readRegister1, readRegister2):
        self.readData1 = self.registers[readRegister1]
        self.readData2 = self.registers[readRegister2]
    

In [187]:
class SignExtender:
    def __init__(self):
        self.data = 0
    def signExtend(self,data):
        if data & 0x8000:
            return data | 0xFFFF0000
        else: return data
        

In [160]:
class Comparator:
    def __init__(self, In1, In2, bne, rst, branch, hold):
        self.In1 = In1
        self.In2 = In2
        self.bne = bne
        self.rst = rst
        self.branch = branch
        self.hold = hold
        self.branchValid = 0
    def compare(self):
        if self.rst:
            self.branchValid = 0
        else:
            if(self.branch and not self.hold and ((self.bne and (self.In1 != self.In2)) or (not self.bne and (self.In1 == self.In2)))):
                self.branchValid = 1
            else:
                branchValid = 0
    

In [188]:
class Decode:
    def __init__(self, cycle, rst, RegWriteEn_WB, jal_WB, instruction, writeData_WB, aluRes_MEM,
                 writeRegister_WB, PCPlus1, forwardA_branch, forwardB_branch, hold):
        self.cycle = cycle
        self.rst = rst
        self.RegWriteEn_WB = RegWriteEn_WB
        self.jal_WB = jal_WB
        self. instruction = instruction
        self.writeData_WB = writeData_WB
        self.aluRes_MEM = aluRes_MEM
        self.writeRegister_WB = writeRegister_WB
        self.PCPlus1 = PCPlus1
        self.forwardA_branch = forwardA_branch
        self.forwardB_branch = forwardB_branch
        self.hold = hold
        self.opCode = instruction>>26 & 0b111111
        self.rs = instruction>>21 & 0b11111
        self.rt = instruction>>16 & 0b11111
        self.rd = instruction >>11 & 0b11111
        self.imm = instruction & 0xFFFF
        self.funct = instruction & 0b111111
        self.shamt = instruction>>6 & 0b11111
        self.jaddress = instruction & 0b1111111111
    def decode(self):
        cu = ControlUnit(opCode=self.opCode, funct=self.funct)
        cu.execute()
        self.RegDst = cu.RegDst
        self.branch = cu.Branch
        self.MemReadEn = cu.MemReadEn
        self.MemtoReg = cu.MemtoReg
        self.ALUOp = cu.ALUOp
        self.MemWriteEn = cu.MemWriteEn
        self.RegWriteEn = cu.RegWriteEn
        self.ALUSrc = cu.ALUSrc
        self.bne = cu.bne
        self.jump = cu.jump
        self.jump = cu.jump
        self.jr = cu.jump
        DestReg = self.rt if not self.RegDst else self.rd
        regAddress = self.writeRegister_WB if not self.jal_WB else 0b11111
        rf = RF()
        rf.readRegisters(readRegister1=self.rs, readRegister2=self.rt)
        self.readData1 = rf.readData1
        self.readData2 = rf.readData2
        rf.writeRegister(rst = self.rst, writeRegister = self.writeRegister_WB,
                         writeData = self.writeData_WB, writeEnable = self.RegWriteEn_WB )
        se = SignExtender()
        self.signExtended = se.signExtend(self.imm)
        self.adderResult = self.PCPlus1 + self.imm
        if(self.forwardA_branch == 0):
            self.fwdA = self.readData1
        elif (self.forwardA_branch == 1):
            self.fwdA = self.aluRes_MEM
        elif (self.forwardA_branch == 2):
            self.fwdA = self.writeData_WB
        else:
            self.fwdA = 0
        if(self.forwardB_branch == 0):
            self.fwdB = self.readData2
        elif (self.forwardB_branch == 1):
            self.fwdB = self.aluRes_MEM
        elif (self.forwardB_branch == 2):
            self.fwdB = self.writeData_WB
        else:
            self.fwdB = 0
        cmp = Comparator(In1 = self.fwdA, In2 = self.fwdB, bne= self.bne, rst = self.rst, branch= self.branch, hold=self.branch)
        cmp.compare()
        self.PCSrc = cmp.branchValid
        

    
        



   



In [199]:
instruction = 0b00010000010000110000000000000101
decode_unit = Decode(
    cycle=1,
    rst=0,
    RegWriteEn_WB=1,
    jal_WB=0,
    instruction=instruction,
    writeData_WB=123,
    aluRes_MEM=456,
    writeRegister_WB=5,
    PCPlus1=4,
    forwardA_branch=0,
    forwardB_branch=0,
    hold=0
)

decode_unit.decode()

print(f"opCode: {decode_unit.opCode}")
print(f"rs: {decode_unit.rs}")
print(f"rt: {decode_unit.rt}")
print(f"rd: {decode_unit.rd}")
print(f"funct: {decode_unit.funct}")
print(f"RegDst: {decode_unit.RegDst}")
print(f"branch: {decode_unit.branch}")
print(f"PCSrc: {decode_unit.PCSrc}")
print(f"imm: {decode_unit.imm}")

opCode: 4
rs: 2
rt: 3
rd: 0
funct: 5
RegDst: 0
branch: 1
PCSrc: 0
imm: 5
